<a href="https://colab.research.google.com/github/VISHAL2077/lab_assignments/blob/main/Assignment_3_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#PART 1
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools
cache = functools.lru_cache(10**6)

In [2]:
class Game:

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, move):
        raise NotImplementedError

    def is_terminal(self, state):
        return not self.actions(state)

    def utility(self, state, player):
        raise NotImplementedError

In [3]:
def play_game(game, strategies: dict, verbose=False):
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose:
            print('Player', player, 'move:', move)
            print(state)
    return state

In [4]:
#PART 2
def minimax_search(game, state):

    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)

infinity = math.inf

In [5]:
def alphabeta_search(game, state):

    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity)

In [15]:
#part 3
class Board:
    def __init__(self, height=3, width=3, to_move='X', utility=0, state=None):
        self.height = height
        self.width = width
        self.to_move = to_move
        self.utility = utility
        self.state = state or {}

    def __getitem__(self, square):
        return self.state.get(square, None)

    def new(self, new_state, to_move):
        state = self.state.copy()
        state.update(new_state)
        return Board(self.height, self.width, to_move, utility=0, state=state)

    def __repr__(self):
        rows = []
        for y in range(self.height):
            row = []
            for x in range(self.width):
                row.append(self.state.get((x, y), '.'))
            rows.append(' '.join(row))
        return '\n'.join(rows)


class TicTacToe(Game):
    def __init__(self, height=3, width=3, k=3):
        self.k = k
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        return self.squares - set(board.state)

    def result(self, board, square):
        player = board.to_move
        new_board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = k_in_row(new_board, player, square, self.k)
        new_board.utility = (0 if not win else +1 if player == 'X' else -1)
        return new_board

    def utility(self, board, player):
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        return board.utility != 0 or len(self.squares) == len(board.state)

    def display(self, board):
        print(board)


def k_in_row(board, player, square, k):
    def in_row(x, y, dx, dy):
        if (x, y) not in board.state or board[x, y] != player:
            return 0
        return 1 + in_row(x + dx, y + dy, dx, dy)

    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))


game = TicTacToe()
board = game.initial
game.display(board)


. . .
. . .
. . .


In [8]:
def random_player(game, state): return random.choice(list(game.actions(state)))

def player(search_algorithm):
    return lambda game, state: search_algorithm(game, state)[1]

In [13]:
#Evaluate the Game Strategy using TicTokToe
play_game(TicTacToe(), dict(X=random_player, O=player(alphabeta_search)), verbose=True).utility

Player X move: (1, 2)
. . .
. . .
. X .
Player O move: (1, 1)
. . .
. O .
. X .
Player X move: (1, 0)
. X .
. O .
. X .
Player O move: (0, 1)
. X .
O O .
. X .
Player X move: (0, 2)
. X .
O O .
X X .
Player O move: (2, 2)
. X .
O O .
X X O
Player X move: (2, 1)
. X .
O O X
X X O
Player O move: (0, 0)
O X .
O O X
X X O


-1

In [14]:
play_game(TicTacToe(), dict(X=player(alphabeta_search), O=player(minimax_search)), verbose=True).utility

Player X move: (0, 1)
. . .
X . .
. . .
Player O move: (2, 1)
. . .
X . O
. . .
Player X move: (1, 2)
. . .
X . O
. X .
Player O move: (0, 0)
O . .
X . O
. X .
Player X move: (1, 1)
O . .
X X O
. X .
Player O move: (1, 0)
O O .
X X O
. X .
Player X move: (2, 0)
O O X
X X O
. X .
Player O move: (0, 2)
O O X
X X O
O X .
Player X move: (2, 2)
O O X
X X O
O X X


0